In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
from ipywidgets import IntProgress
from IPython.display import display
import os
from IPython.display import clear_output
import json
from selenium.webdriver.support import expected_conditions
from sc_header import aplanar, aplanar_lista,createDriver, extract_json_objects

In [2]:
# def createDriver():

# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'    

# chrome_options = Options()  
# chrome_options.add_argument("--headless")  
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument(f'user-agent={user_agent}')
# chrome_options.add_argument("--window-size=1825x1644")
# chromeOptions.addArguments("--disable-popup-blocking")
# chromeOptions.addArguments("--disable-notifications")
# chromeOptions.addArguments("--window-size=1920,1080")

# browser = webdriver.Chrome('/usr/bin/chromedriver', options = chrome_options)
browser = createDriver()
browser.delete_all_cookies()

In [3]:
url_base = 'https://www.shein.com/daily-new.html?cat_ids=2030,2032&ici=www_tab01navbar01&scici=navbar_2~~tab01navbar01~~1~~webLink~~~~0~~0'

browser.get(url_base)

In [4]:
#browser.save_screenshot('asd.png')


In [5]:
#CLICK EN MODAL
try:
    browser.find_element_by_css_selector('body > div.c-outermost-ctn > div.c-vue-coupon > div.coupon-mask.she-scroll-hide > div > i').click()
except:
    pass

In [6]:
#browser.find_element_by_css_selector('.nav2-sec-ctn.j-nav2-sec-ctn.j-sa-nav2-sec-ctn').click()

In [7]:
fecha = browser.find_element_by_css_selector('.filter-title.level-title.leftnav-second-title').find_element_by_tag_name('a')
fecha_alta = fecha.text
fecha.click()

In [8]:
women_element = browser.find_element_by_css_selector('.list-classify-filter.list-indent-one.j-fold-category-ctn').find_element_by_tag_name('li').find_element_by_tag_name('a')
women = women_element.text.upper()

In [9]:
#CLICK EN WOMEN
women_element.click()

In [10]:
category_list = browser.find_elements_by_css_selector('.list-classify-filter.list-indent-one')[1]

In [11]:
html_cat = []

for cat in category_list.find_elements_by_tag_name('li'):
    
    html_cat.append([cat.find_element_by_tag_name('a').get_attribute('href'),
                     cat.find_element_by_tag_name('a').get_attribute('data-cat-url-name')])
    

In [12]:
LINK_CATEGORIAS = [i for i in html_cat if i[1] in ['Clothing','Lingerie & Loungewear','Beachwear','Activewear']]

In [13]:
import pickle
def agregarLinks(url_aux,driver,lista,tipologia_aux,url_concat):
    time.sleep(1)
    driver.get(url_aux)
#     pickle.dump(browser.get_cookies() , open("cookies.pkl","wb"))
#     cookies = pickle.load(open("cookies.pkl", "rb"))
#     for cookie in cookies:
#         browser.add_cookie(cookie)
    
    for elements in browser.find_elements_by_css_selector('.list-line-decorate.j-attr-filter'):
        
        if elements.find_element_by_tag_name('li').text == 'Color':
        #if elements.text == 'Color':
            print(tipologia_aux)

            for color in elements.find_elements_by_class_name('j-auto-attrlink'):

                lista.append([tipologia_aux,
                              color.get_attribute('data-attrvalue'),
                              color.get_attribute('href')])
                
    return lista

In [14]:
Tlistas = []
browser = createDriver()
url_base = 'https://www.shein.com'
for lsc in LINK_CATEGORIAS:
    
    agregarLinks(lsc[0],
                 browser,
                 Tlistas,
                 lsc[1],
                 url_base)

Clothing
Lingerie & Loungewear
Beachwear
Activewear


In [15]:
#browser.delete_all_cookies()

In [16]:
hrefs_list = []
ex_col = []

for lista in Tlistas:
    
#for lista in Tlistas[4:]:

    try:
        browser.delete_all_cookies()
        browser.get(lista[2])
    except:
        pass
    
    while True:
        
        soup = BeautifulSoup(browser.page_source,'html.parser')

        indice_= 0
        
        if soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a') == []: raise
            
        for element in soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a'):
        #for element in soup.find_all(class_='c-goodsitem__ratioimg j-item-msg j-item-msg-a j-expose__target-goods-img'):
            
            
            href_aux = url_base + element.get('href')

            hrefs_list.append([indice_,
                               lista[0],
                               lista[1],
                               lista[2],
                               href_aux
                              ])
            indice_+=1
                
            #except:
            #    raise
            #    ex_col.append([lista[0],lista[2],lista[1]])
            #    pass            
        try:
            browser.find_element_by_class_name('page-next').find_element_by_tag_name('a').click()
        except:
            break
#browser.quit()


In [17]:
#soup.find_all(class_='c-goodsitem__ratioimg j-item-msg j-item-msg-a j-expose__target-goods-img')

In [37]:
df = pd.DataFrame(hrefs_list,columns=['pos','tipo','color','url_scraper','href'])

In [38]:
df['tipo'].unique()

array(['Clothing', 'Lingerie & Loungewear', 'Beachwear', 'Activewear'],
      dtype=object)

In [39]:
def check_length(serie):
    
    #if menor a 8 es porque hay un link donde el max es un numero que no es el id_producto
    ##caso especial : 'https://www.shein.com/skdress07191118781-p-929597-cat-2005.html?scici=homepage_164~~0_Banner_1_0_hotZone_w7bymbmsz~~3_2~~real_1991~~ccc_shein_pc_kids-homepage_default~~0~~50001'
    #
    return str(max([X for X in map(int,re.findall(r"([\d]+)",serie)) if len(str(X))<8]))
    

In [40]:
df['auxi'] = df['href'].apply(check_length)

In [41]:
len(df.drop_duplicates()),len(df)

(445, 445)

In [42]:
#active = [i for i in Tlistas if i[0] == 'Activewear']

In [43]:
if len(df['tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [44]:
len(df['tipo'].unique().tolist()),len(set([item[0] for item in Tlistas]))

(4, 4)

In [26]:
def scrape_url(url_aux,pos_aux,url_scraper,aux_tipologia,aux_color,aux_flag):
    
    lista_auxiliar = []
    url = url_aux
    for _ in range(5):
        try:
            txt = re.findall(r'goodsInfo\s*:\s*({.*})', requests.get(url).text)
            txt = txt[0]
            break
        except:
            pass
    try:
        data = json.loads(txt)
    except:
        return [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

    #DESCRIPCION
    descripcion = (data['detail']['goods_name'])
    #PRECIO 

    precio = data['price']['info']['price'][aux_flag]['retailPrice']['amount']
    #PRECIO CON DTO
    precio_dto = data['price']['info']['price'][aux_flag]['salePrice']['amount']    
    #ID
    codigo = data['localSize']['skc']
    #JPG
    jpg = data['goods_imgs']['main_image']['origin_image']

    for item in data['attrSize']:

        lista_auxiliar.append([pos_aux,
                            aux_tipologia,
                            aux_color,
                            codigo,
                            descripcion,
                            item['attr_value_en'],
                            item['stock'],
                            precio,
                            precio_dto,
                            jpg,
                            url_aux,
                            url_scraper
                            ])
    return lista_auxiliar

def scrape_batch(url_chunk):
    chunk_resp = []
    for indices,fila in url_chunk.iterrows():
        chunk_resp.append(scrape_url(fila['href'],fila['pos'],fila['url_scraper'],fila['tipo'],fila['color'],fila['auxi']))
                     
    return chunk_resp


In [27]:
start_ = datetime.datetime.now()
items = []

batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

var = 0
for url_chunk in url_chunks:
    items.append(scrape_batch(url_chunk))
    var += 1
    
    if var == 30:
        var = 0
        try:
            df = pd.DataFrame(items)
            df.to_excel('lanzamiento.xlsx')
            
                                                                                            
            file = open("lanzamiento.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Erro al crear .xlsx")
        
    
end_ = datetime.datetime.now()

In [28]:
new_list = aplanar(items)

In [29]:
print('Tiempo de ejecución: {}'.format(end_ - start_)[:-4])

Tiempo de ejecución: 1:53:03.83


In [30]:
new_list =[ i for i in new_list if type(i) != int]

In [31]:
df = pd.DataFrame(new_list)

In [32]:
df.rename(columns={0:'pos',
                  1:'tipo',
                  2:'color',
                  3:'sku',
                  4:'descripcion',
                  5:'talle',
                  6:'stock',
                  7:'precio',
                  8:'precio_dto',
                  9:'img',
                  10:'url',
                  11:'pagina_scraper'
                  },inplace=True)
df['fecha_alta'] = fecha_alta
df["marca"] = "SHEIN"
df["moneda"] = "DOLAR US$"

In [35]:
browser.save_screenshot('asd.png')

True

In [33]:
df["precio_dto"] = (df["precio_dto"]
                .str.extract(r"([\d,\.]+)", expand=False)
                .str.replace(",", ".")
                .astype(float))

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
df["precio"] = (df["precio"]
                .str.extract(r"([\d,\.]+)", expand=False)
                .str.replace(",", ".")
                .astype(float))

In [ ]:
df['sexo'] ='Mujer'

In [ ]:
df = df.drop_duplicates()

In [ ]:
browser.quit()

In [ ]:
df.dropna(inplace=True)

In [ ]:
set([i[1] for i in new_list])

In [ ]:
fecha = datetime.date.today()
df.to_excel('../Salida/lanzamiento'+str(fecha)+'.xlsx')
# writer = ExcelWriter('../Salida/lanzamiento'+str(fecha)+'.xlsx')
# df.to_excel(writer,'Hoja1')
# writer.save()

In [ ]:
raise

In [40]:
browser.quit()

In [ ]:
len(df)

In [41]:
import pathlib
pathlib.Path().absolute()

PosixPath('/home/aa/cloudJ/JB/PRD/scrapper/Scripts')

In [42]:
import os
os.getcwd()

'/home/aa/cloudJ/JB/PRD/scrapper/Scripts'

In [ ]:

    window.goodsDetailv2SsrData = {
        goodsDetailv2Language: {"lang_category_percent_sale":"Buy {0} Get 1 {1} Off","lang_category_percent_sale_all":"BUY {0} GET {1}% OFF","lang_category_percent_free":"Buy {0} Get 1 Free","lang_footer_shipping_info":"Shipping Info","lang_footer_return_policy":"Return Policy","lang_footer_freetrial":"Free Trial Report","SHEIN_PC_KEY_4177":"Size Guide","SHEIN_PC_KEY_5051":"Please enter 6 digit pincode to check delivery&services.","lang_goods_customer_view":"Customers Also Viewed","lang_goods_flash_back_to_flashsale":"Back to flash sale","lang_goods_flash_sold":"{0} SOLD","lang_goods_flash_end_in":"ENDS IN {0}","SHEIN_PC_KEY_3219":"Flash Sale","SHEIN_PC_KEY_5092":"Ends in {0}","lang_goods_description":"Description","lang_goods_stock":"In Stock","lang_goods_out_ofstock":"Out of Stock","lang_goods_limit_buy":"Max Qty allowed for this limited product is <span class=\"red\">{0}</span>.","lang_goods_word_color":"Color","lang_goods_get_the_look":"Get The Look","add_to_bag":"Add To Bag","lang_goods_true_to_sze":"True to Size","lang_goods_large":"Large","lang_goods_small":"Small","added":"ADDED!","reviews_size":"size","reviews_image":"Image","reviews_all_all":"All","reviews_with_picture":"With Picture","reviews_no_picture":"No Picture","reviews_recent_to_oldest":"Most Recent to Oldest","reviews_oldest_to_most_recent":"Oldest to Most Recent","reviews_likes_high_to_low":"Likes High to Low","lang_goods_own":"Customer Reviews","lang_goods_review_average":"Average Rating","lang_review_fitwell":"Did the item fit well?","reviews_all_reviews":"Reviews","reviews_sort_by":"Sort by","no_comment_tips":"No comments on this product found based on your request.","lang_goods_item_size":"Item Size:","lang_goods_overall_fit":"Overall Fit:","lang_goods_view_all":"View All","lang_goods_bra_size":"Bra Size:","lang_goods_bust_size":"Bust:","lang_goods_weight":"Weight:","lang_goods_height":"Height:","lang_goods_hips":"Hips:","lang_goods_waist":"Waist:","lang_goods_reply":"Answer","lang_goods_best_sellers":"Best Sellers","lang_size_fit":"Size & Fit","lang_goods_style_gallery":"Style Gallery","lang_style_gallery_empty_p1":"The Style Gallery is a great new way for you to share outfit photos. It's meant to show all the expressive, creative and inspirational personalities!","lang_style_gallery_empty_p2":"Earn <em>20 points</em> by uploading your own photo.","lang_goods_upload_photo":"Upload your Photo","lang_bfs_view_photo":"add your photo","SHEIN_PC_KEY_3894":"Details","lang_cart_color":"Color","SHEIN_PC_KEY_4923":"Estimated to be delivered on {0}.","lang_goods_may_like":"You May Also Like","choose_my_size":"choose my size ","choose_my_size_tips":"according to the information you enter, a size will be recommended with other customers' feedback on the size, which you can use for your reference.","find_my_size":"Find my size","sorry":"sorry","sorry_no_recommend_size":"No size can be recommended. <br/>","similar_shapes_chose":"Most customers who have similar shapes chose","continue_shopping":"Continue shopping","edit_my_size":"Edit My Size","thin":"Thin","average":"Average","thick":"Thick","goods_add_size":"Add Size {0}","SHEIN_PC_KEY_4535":"Other Options","lang_category_view_all":"View All","lang_goods_learn":"Learn more","size_recommended":"Size Recommended","check_my_size":"Check my size","SHEIN_PC_KEY_4903":"NEW","SHEIN_PC_KEY_4927":"Hurry! Almost Sold Out...","SHEIN_PC_KEY_4645":"Approx {0} - {1}","SHEIN_PC_KEY_4431":"Free Standard Shipping","SHEIN_PC_KEY_4425":"Free standard shipping on orders over {0}","SHEIN_PC_KEY_4424":"Free Express Shipping","SHEIN_PC_KEY_4426":"Free express shipping on orders over {0}","SHEIN_PC_KEY_4432":"Free Flat Rate Shipping","SHEIN_PC_KEY_4433":"Free flat rate shipping on orders over {0}","free_shipping":"Free shipping","free_ship_over":"Free shipping on orders over {0}","lang_goods_flash_lighting_deals":"flash sale","lang_goods_only_one":"purchase limit exceeded","SHEIN_PC_KEY_3274":"You have reached the Max Qty of flash sale","lang_goods_earn":"Earn","lang_goods_points":"SHEIN Points","SHEIN_PC_KEY_4889":"Do you like this product?","SHEIN_PC_KEY_4850":"Find Similar","SHEIN_PC_KEY_5147":"Show Similar","SHEIN_PC_KEY_5150":"This product is sold out, please check out our similar items.","lang_goods_out":"Sold Out","lang_goods_size_size":"Size","lang_goods_size_size1":"Size","lang_goods_please_select_size":"Please Select Size","lang_goods_chart":"Size Chart","SHEIN_PC_KEY_4930":"Return and exchange goods are not supported.","SHEIN_PC_KEY_5082":"Item Size","SHEIN_PC_KEY_4929":"Free Return & Exchange","SHEIN_PWA_KEY_3981":"Related Searches","lang_goods_years":"Years","SHEIN_PC_KEY_5088":"User Size","SHEIN_PC_KEY_5090":"LOOK","SHEIN_PC_KEY_5089":"All Reviews","lang_goods_home":"Home","SHEIN_PC_KEY_4437":"Model Wear","lang_list_please_select":"Recommend","lang_categoryt_off":"off","lang_show_have_clicked":"You have already liked it.","lang_member_wallet_failed":"Failed","SHEIN_PC_KEY_3670":"Preference","lang_member_shipping_method":" Shipping Method ","lang_member_out_of_stock":"Out of Stock","lang_member_how_to_measure":"How to measure","lang_member_chart_explain":"This size chart is for reference purposes only. Note that sizing will vary between brands.","lang_member_shoe_explain":"Not all shoe selections will carry all sizes.","lang_member_foot_length":"Length","lang_member_word_delivery":"Delivery","lang_member_size_height":"Height:","lang_member_size_bust_size":"Bust:","lang_member_size_waist":"Waist:","lang_member_size_hips":"Hips:","SHEIN_PC_KEY_3166":"Free Shipping","goods_limit_buy":"Max Qty allowed for this limited product is {0}","SHEIN_PC_KEY_3243":"{0} off over {1}+","SHEIN_PC_KEY_3244":"{0} off over {1}+ items","SHEIN_PC_KEY_3237":"Add-on items","SHEIN_PC_KEY_3238":"Order over {0} can purchase items at {1}","SHEIN_PC_KEY_3378":"Return coupons","SHEIN_PC_KEY_3375":"Gifts rewarded","SHEIN_PC_KEY_3374":"Buy more save more","SHEIN_PC_KEY_3249":"{0} for {1} items","SHEIN_PC_KEY_3250":"{0} for per {1} items","SHEIN_PC_KEY_3568":"2nd {0} 3rd {1}","SHEIN_PC_KEY_3567":"2nd {0} off 3rd {1} off","SHEIN_PC_KEY_3565":"2nd {0} off","SHEIN_PC_KEY_3566":"2nd {0}","SHEIN_PC_KEY_3561":"Buy {0} Get {1} coupons","SHEIN_PC_KEY_3564":"Buy {0} Get free shipping coupons","SHEIN_PC_KEY_4375":"Buy {0} Get 1 {1}% off","SHEIN_PC_KEY_4293":"BUY  {0}  GET 1 {1}%  OFF","SHEIN_PC_KEY_4294":"Buy {0} Get {1} {2}% off","SHEIN_PC_KEY_5056":"Buy {0} get one special discount {1} % off","SHEIN_PC_KEY_5057":"Buy {0} get {1} for a special discount {2}% off","SHEIN_PC_KEY_5058":"Buy {0} get special discount {1}% off","SHEIN_PC_KEY_5059":"Second item get special discount {0}% off","SHEIN_PC_KEY_5060":"2nd get special discount {0}% off, 3rd get speical discount {1}% off","SHEIN_PC_KEY_3905":"Only {0} Left","clearCartTips":"Please review your shopping cart as the item count has reached the limit.","SHEIN_PC_KEY_4191":"This item cannot be returned due to promotional activities","SHEIN_PC_KEY_4077":"RETURN POLICY","lang_shipping_time":"Shipping Time","SHEIN_PC_KEY_4204":"Costs","SHEIN_PC_KEY_4198":"Shipping to","SHEIN_PC_KEY_3256":"Business Days","lang_daily_word_more":"More"},
        versionAbtInfo: {"type":"expgroup","bid":"4715","eid":566,"p":"newnew","posKey":"SPcItemdetail","poskey":"SPcItemdetail","sceneStr":"Itemdetail"},
        productIntroData: {"getNewFlagStatus":false,"multiLocalSize":{"category_id":1738,"category_name":"T-Shirts","supplier_category_id":59,
                                                                      "skc":"swtee07191221869","size_rule_list":{"DE":[{"name":"S","correspond":"36"},
                                                                                                                       {"name":"XS","correspond":"34"},
                                                                                                                       {"name":"M","correspond":"38"},
                                                                                                                       {"name":"L","correspond":"40/42"},
                                                                                                                       {"name":"XL","correspond":"44"},{"name":"XXL","correspond":"46"}],"EU":[{"name":"S","correspond":"36"},{"name":"XS","correspond":"34"},{"name":"M","correspond":"38"},{"name":"L","correspond":"40/42"},{"name":"XL","correspond":"44"},{"name":"XXL","correspond":"46"}],"UK":[{"name":"S","correspond":"8"},{"name":"XS","correspond":"6"},{"name":"M","correspond":"10"},{"name":"L","correspond":"12/14"},{"name":"XL","correspond":"16"},{"name":"XXL","correspond":"18"}],"AU":[{"name":"S","correspond":"8"},{"name":"XS","correspond":"6"},{"name":"M","correspond":"10"},{"name":"L","correspond":"12/14"},{"name":"XL","correspond":"16"},{"name":"XXL","correspond":"18"}],"US":[{"name":"S","correspond":"4"},{"name":"XS","correspond":"2"},{"name":"M","correspond":"6"},{"name":"L","correspond":"8/10"},{"name":"XL","correspond":"12"},{"name":"XXL","correspond":"14"}],"IT":[{"name":"XS","correspond":"38"},{"name":"S","correspond":"40"},{"name":"M","correspond":"42"},{"name":"L","correspond":"44/46"},{"name":"XL","correspond":"48"},{"name":"XXL","correspond":"50"}]},"goods_id":979392},"detail":{"goods_id":"979392","cat_id":"1738","goods_sn":"swtee07191221869","goods_url_name":"Drop Shoulder Figure & Slogan Graphic Top","goods_thumb":"//img.ltwebstatic.com/images3_pi/2020/03/02/1583144430d13e817e25cefc40dcffad8b85280a5b_thumbnail_220x293.jpg","original_img":"//img.ltwebstatic.com/images3_pi/2020/03/02/1583144430d13e817e25cefc40dcffad8b85280a5b.jpg","goods_img":"//img.ltwebstatic.com/images3_pi/2020/03/02/1583144430d13e817e25cefc40dcffad8b85280a5b_thumbnail_405x552.jpg","is_stock_enough":"0","is_virtual_stock":"0","supplier_id":"1223","goods_desc":"","goods_detail_desc":"","goods_name":"Drop Shoulder Figure & Slogan Graphic Top","brand":"SHEIN","sizeTemplate":{"image_url":"http://img.ltwebstatic.com/images2_pi/2018/05/28/15274953862701810197.png","description_multi":[{"sort":1,"name":"Bust","description":"Measure under your arms around the fullest part of your bust. Make sure to keep the tape level!"},{"sort":2,"name":"Waist","description":"Make sure the measuring tape fits comfortably as you measure around the narrowest part of your natural waist."},{"sort":3,"name":"Hips","description":"Stand with your feet together, and measure around the widest part of your hips."}]},"supplier_top_category_id":"59"…